In [ ]:
#import all necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix

from implicit.als import AlternatingLeastSquares
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import GPT2Tokenizer
from sklearn.cluster import KMeans
from typing import List, Dict, Tuple
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2Model
from sklearn.model_selection import train_test_split
from typing import List, Dict, Tuple


mount google  drive

In [61]:
#Get the data into the workspace
Products_train =pd.read_csv('/content/drive/MyDrive/Few shot learning for categorization and recommendation/products_train.csv')

Sessions_train =pd.read_csv('/content/drive/MyDrive/Few shot learning for categorization and recommendation/sessions_train.csv')

In [4]:
#check a sample of the product data
Products_train.head()

,id,locale,title,price,brand,color,size,model,material,author,desc
0,B005ZSSN10,DE,RED DRAGON Amberjack 3 - Steel Tip 22 Gramm Wo...,30.95,RED DRAGON,NaN,NaN,RDD0089,NaN,NaN,Amberjacks Steel Dartpfeile sind verfügbar in ...
1,B08PRYN6LD,DE,Simply Keto Lower Carb* Schokodrops ohne Zucke...,17.90,Simply Keto,NaN,750 g (1er Pack),NaN,NaN,NaN,🌱 NATÜRLICHE SÜSSE DURCH ERYTHRIT - Wir stelle...
2,B09MBZJ48V,DE,"Sennheiser 508377 PC 5.2 Chat, Stilvolles Mult...",68.89,Sennheiser,Multi-Colour,One size,508377,Kunstleder,NaN,3.5 MM BUCHSE - Kann problemlos an Geräte mit ...
3,B08ZN6F26S,DE,AmyBenton Auto ab 1 2 3 ahre - Baby Aufziehbar...,18.99,Amy & Benton,Animal Car,NaN,2008B,aufziehauto 1 jahr,NaN,【Auto aufziehbar】: Drücken Sie einfach leicht ...
4,B094DGRV7D,DE,PLAYMOBIL - 70522 - Cavaliere mit grauem Pony,7.17,PLAYMOBIL,Nicht Zutreffend.,OneSize,70522,Polypropylen,NaN,Inhalt: 1 Stück


In [5]:
# check a sample of the session data
Sessions_train.head()

,prev_items,next_item,locale
0,['B09W9FND7K' 'B09JSPLN1M'],B09M7GY217,DE
1,['B076THCGSG' 'B007MO8IME' 'B08MF65MLV' 'B001B...,B001B4THSA,DE
2,['B0B1LGXWDS' 'B00AZYORS2' 'B0B1LGXWDS' 'B00AZ...,B0767DTG2Q,DE
3,['B09XMTWDVT' 'B0B4MZZ8MB' 'B0B7HZ2GWX' 'B09XM...,B0B4R9NN4B,DE
4,['B09Y5CSL3T' 'B09Y5DPTXN' 'B09FKD61R8'],B0BGVBKWGZ,DE


# **Data description.**

user sessions: User sessions are a list of products that a user has engaged with in chronological order

product attributes: product title, price in local currency, brand, colour, and description.



# **Colum Descriptions**

**locale**: the locale code of the product (e.g., DE)

**id:** a unique for the product. Also known as Amazon Standard Item Number (ASIN) (e.g., B07WSY3MG8)

**title:** title of the item (e.g., “Japanese Aesthetic Sakura Flowers Vaporwave Soft Grunge Gift T-Shirt”)

**price:** price of the item in local currency (e.g., 24.99)

**brand:** item brand name (e.g., “Japanese Aesthetic Flowers & Vaporwave Clothing”)

**color:** color of the item (e.g., “Black”)

**size:**size of the item (e.g., “xxl”)

**model:**model of the item (e.g., “iphone 13”)

**material:** material of the item (e.g., “cotton”)

**author:** author of the item (e.g., “J. K. Rowling”)

**desc:** description about a item’s key features and benefits called out via bullet points (e.g., “Solid colors: 100% Cotton; Heather Grey: 90% Cotton, 10% Polyester; All Other Heathers …”

In [6]:
#check unique locales
Products_train['locale'].unique()

array(['DE', 'JP', 'UK', 'ES', 'FR', 'IT'], dtype=object)

**Findings**

In [7]:
#check the general info
Products_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1551057 entries, 0 to 1551056
Data columns (total 11 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   id        1551057 non-null  object 
 1   locale    1551057 non-null  object 
 2   title     1551049 non-null  object 
 3   price     1551057 non-null  float64
 4   brand     1531686 non-null  object 
 5   color     1125428 non-null  object 
 6   size      917091 non-null   object 
 7   model     761135 non-null   object 
 8   material  834382 non-null   object 
 9   author    73509 non-null    object 
 10  desc      1424083 non-null  object 
dtypes: float64(1), object(10)
memory usage: 130.2+ MB


In [8]:
#check general info of session data
Sessions_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3606249 entries, 0 to 3606248
Data columns (total 3 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   prev_items  object
 1   next_item   object
 2   locale      object
dtypes: object(3)
memory usage: 82.5+ MB


# **Data cleaning**

In [9]:
#check if we have the right data types
Products_train.dtypes

,0
id,object
locale,object
title,object
price,float64
brand,object
color,object
size,object
model,object
material,object
author,object


we have the datasets in the correct formats

In [10]:
#check null value counts
Products_train.isnull().sum()

,0
id,0
locale,0
title,8
price,0
brand,19371
color,425629
size,633966
model,789922
material,716675
author,1477548


In [11]:
#check null values in session training data
Sessions_train.isnull().sum()

,0
prev_items,0
next_item,0
locale,0


we have to handle these values with care,
let check total length of the dataset

In [12]:
#length of entire dataset
len(Products_train)

1551057

approximately 1.5M this means
1. The author column has so many missing values it's better if i get rid of it
2.the remaining columns has missing values in the thousands, most of this columns are important for training

my initial decision is
1. drop the author column
2. drop all rows with missing values

In [13]:
#drop the author column
Products_train.drop('author',axis=1,inplace=True)

In [14]:
#drop any row that contain a missing value
Products_train.dropna(inplace=True)

In [15]:
#check product info
Products_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 269581 entries, 2 to 1551054
Data columns (total 10 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id        269581 non-null  object 
 1   locale    269581 non-null  object 
 2   title     269581 non-null  object 
 3   price     269581 non-null  float64
 4   brand     269581 non-null  object 
 5   color     269581 non-null  object 
 6   size      269581 non-null  object 
 7   model     269581 non-null  object 
 8   material  269581 non-null  object 
 9   desc      269581 non-null  object 
dtypes: float64(1), object(9)
memory usage: 22.6+ MB


In [16]:
#check first ten values
Products_train.head(10)

,id,locale,title,price,brand,color,size,model,material,desc
2,B09MBZJ48V,DE,"Sennheiser 508377 PC 5.2 Chat, Stilvolles Mult...",68.89,Sennheiser,Multi-Colour,One size,508377,Kunstleder,3.5 MM BUCHSE - Kann problemlos an Geräte mit ...
4,B094DGRV7D,DE,PLAYMOBIL - 70522 - Cavaliere mit grauem Pony,7.17,PLAYMOBIL,Nicht Zutreffend.,OneSize,70522,Polypropylen,Inhalt: 1 Stück
5,B09JNNBDH5,DE,"URBZUE Handwärmer, 10000mAh USB aufladbar und ...",20.99,URBZUE,Rosagold,13.5*9*5,Oro rosa,Aluminium,Einstellbarer Temperaturmodus: Aufladbare Hand...
9,B0B9MPKYJK,DE,Amstory Stirnlampe LED Wiederaufladbar Joggen ...,25.99,Amstory,Schwarz,2 Stücke,LMHL-0006BK,ABS,☀【1000 Lumen Superheller Kopflampe】 Der LED He...
16,B07K27BTM8,DE,"JewelCandle ""Magical Winter Duftkerze mit Über...",19.99,JewelCandle,Creme,Boucles d'oreilles,123456,Wachs,"JewelCandle ""Magical Winter Duftkerze mit Über..."
28,B0868TBNZZ,DE,"Audi collection Isolierbecher, 1 Stück (1er Pack)",37.90,Audi Sport GmbH,Silber,1 Stück (1er Pack),3292000100,Kunststoff,Audi Isolierbecher aus Edelstahl - Doppelwandi...
33,B09QMCVW6P,DE,"BOSS Herren R-Neck T-Shirt, 3er Pack Classic, ...",31.65,BOSS,New - Open Miscellaneous 984,L,50475284,100% Baumwolle,"100% Baumwolle, höchster Tragecomfort und opti..."
34,B09JCC7FVK,DE,NIULAA 2er Set Wolle Kissenbezug 50x50 cm Weiß...,19.68,NIULAA,Weiß,50 x 50 cm,B-White,Wolle,KISSENBEZUG MIT VERSTECKTER REISSVERSCHLUSS - ...
40,B0009585PI,DE,"Trixie 2288 Dog Activity Trainings-Discs, ø 4,...",2.99,TRIXIE,Silber,1 Stück (1er Pack),4011905022888,Metall,Effektive Ausbildung mit der Signalton-Methode
48,B09L85HNCT,DE,KETEEN Glitzer Handyhülle für Samsung Galaxy S...,13.98,Keteen,Rosa,Samsung Galaxy S20 FE,Samsung Galaxy S20 FE,Silikon,❌❌ OHNE Integriertem Displayschutz Schutzhülle...


For each of the different locales let's check the total number of values

In [17]:
#check the datasize for each locale
Products_train['locale'].value_counts()

,count
locale,
DE,91358
UK,89744
JP,62318
ES,9021
FR,8680
IT,8460


1. Technique number one is to squash everything together and pray that the model during the tokenization and emb stages is able to put the similar words from different languages in the same vector space

2. build different models for each of the languages and put them together

3. look at what other people did

## Possible challenges.
1. since we have dropped some of the products it will be difficult to point out some of the products interacted with for instance given, a series of products in the past, the next product interacted with might not be in the data, we are also supposed to use these session data to determine which products are less interacted with

# **Exploratory Data analysis**

 A simple walkthrough of the data, for instance, take first five users,
 check the products they interacted with and the next product they interracted with for a trend

In [18]:
Sessions_train.head()

,prev_items,next_item,locale
0,['B09W9FND7K' 'B09JSPLN1M'],B09M7GY217,DE
1,['B076THCGSG' 'B007MO8IME' 'B08MF65MLV' 'B001B...,B001B4THSA,DE
2,['B0B1LGXWDS' 'B00AZYORS2' 'B0B1LGXWDS' 'B00AZ...,B0767DTG2Q,DE
3,['B09XMTWDVT' 'B0B4MZZ8MB' 'B0B7HZ2GWX' 'B09XM...,B0B4R9NN4B,DE
4,['B09Y5CSL3T' 'B09Y5DPTXN' 'B09FKD61R8'],B0BGVBKWGZ,DE


In [19]:
#let's pick a the second prev_item and next_item in the session data
sample=Sessions_train.iloc[1]
sample.reset_index(drop=True, inplace=True)


In [20]:
prev_item=sample[0]
next_item=sample[1]

to be able to pick a product and search, set the id as index in product_train


In [ ]:
# set id as index
Products_train.set_index('id',inplace=True)


In [23]:
Products_train.head()

,locale,title,price,brand,color,size,model,material,desc
id,,,,,,,,,
B09MBZJ48V,DE,"Sennheiser 508377 PC 5.2 Chat, Stilvolles Mult...",68.89,Sennheiser,Multi-Colour,One size,508377,Kunstleder,3.5 MM BUCHSE - Kann problemlos an Geräte mit ...
B094DGRV7D,DE,PLAYMOBIL - 70522 - Cavaliere mit grauem Pony,7.17,PLAYMOBIL,Nicht Zutreffend.,OneSize,70522,Polypropylen,Inhalt: 1 Stück
B09JNNBDH5,DE,"URBZUE Handwärmer, 10000mAh USB aufladbar und ...",20.99,URBZUE,Rosagold,13.5*9*5,Oro rosa,Aluminium,Einstellbarer Temperaturmodus: Aufladbare Hand...
B0B9MPKYJK,DE,Amstory Stirnlampe LED Wiederaufladbar Joggen ...,25.99,Amstory,Schwarz,2 Stücke,LMHL-0006BK,ABS,☀【1000 Lumen Superheller Kopflampe】 Der LED He...
B07K27BTM8,DE,"JewelCandle ""Magical Winter Duftkerze mit Über...",19.99,JewelCandle,Creme,Boucles d'oreilles,123456,Wachs,"JewelCandle ""Magical Winter Duftkerze mit Über..."


In [26]:
Products_train.loc['B0B9MPKYJK']

,B0B9MPKYJK
locale,DE
title,Amstory Stirnlampe LED Wiederaufladbar Joggen ...
price,25.99
brand,Amstory
color,Schwarz
size,2 Stücke
model,LMHL-0006BK
material,ABS
desc,☀【1000 Lumen Superheller Kopflampe】 Der LED He...


from  a couple of errors encountered, the prev_items in the session data is seen as a string and not a list of strings, this means we need to clean it else, we lose the information it represents

In [33]:
#example
type(prev_item)

str

we will need to
1. create a function that will be mapped to the whole prev_item column to turn it into a viable list of strings

In [51]:
def parse_list_string(s):
    # Remove brackets and split by space
    cleaned = s.strip('[]').strip()
    # Split by space and remove any quotes
    items = [item.strip("'").strip('"') for item in cleaned.split()]
    return items

parse_list_string(prev_item)

['B076THCGSG', 'B007MO8IME', 'B08MF65MLV', 'B001B4TKA0']

In [53]:
prev_item=parse_list_string(prev_item)

In [57]:
#write a function that accepts a string and uses it to index the product to find the product title
def get_title(id):
  if id in Products_train.index:
    return Products_train.loc[id]['title']
  else:
    return 'product not found'

get_title(prev_item[1])

'Caparol Capa DIN 12,500 L Wandfarben'

check the products in prev_item and the corresponding next item interracted with


In [60]:
for item in prev_item:
  print(f"Item: {item}: {get_title(item)}")
print(f"Next item: {get_title(next_item)}")

Item: B076THCGSG: product not found
Item: B007MO8IME: Caparol Capa DIN 12,500 L Wandfarben
Item: B08MF65MLV: product not found
Item: B001B4TKA0: product not found
Next item: product not found


This is a problem with dropping all 'NAN' values
Therefore i will need to go back and keep the 'NAN' drop the author column and handle "Nan" values during preprocessing, and also use a model with good NAN value handling

Next Steps
1. keep missing values and drop author column
2. apply string_parsing function to the prev_item column
3. Skip feature Engineering and perform firts training

# **Checkpoint**
reload the data and apply the steps below then move to preprocessing

In [62]:
#drop author column
Products_train.drop('author',axis=1,inplace=True)

#this function converts the prev_item in session data to the right format
def parse_list_string(s):
    # Remove brackets and split by space
    cleaned = s.strip('[]').strip()
    # Split by space and remove any quotes
    items = [item.strip("'").strip('"') for item in cleaned.split()]
    return items

#apply this function to the prev_item in the session column
Sessions_train['prev_items'] = Sessions_train['prev_items'].apply(parse_list_string)

In [64]:
#check the products data now
Products_train.head()

,id,locale,title,price,brand,color,size,model,material,desc
0,B005ZSSN10,DE,RED DRAGON Amberjack 3 - Steel Tip 22 Gramm Wo...,30.95,RED DRAGON,NaN,NaN,RDD0089,NaN,Amberjacks Steel Dartpfeile sind verfügbar in ...
1,B08PRYN6LD,DE,Simply Keto Lower Carb* Schokodrops ohne Zucke...,17.90,Simply Keto,NaN,750 g (1er Pack),NaN,NaN,🌱 NATÜRLICHE SÜSSE DURCH ERYTHRIT - Wir stelle...
2,B09MBZJ48V,DE,"Sennheiser 508377 PC 5.2 Chat, Stilvolles Mult...",68.89,Sennheiser,Multi-Colour,One size,508377,Kunstleder,3.5 MM BUCHSE - Kann problemlos an Geräte mit ...
3,B08ZN6F26S,DE,AmyBenton Auto ab 1 2 3 ahre - Baby Aufziehbar...,18.99,Amy & Benton,Animal Car,NaN,2008B,aufziehauto 1 jahr,【Auto aufziehbar】: Drücken Sie einfach leicht ...
4,B094DGRV7D,DE,PLAYMOBIL - 70522 - Cavaliere mit grauem Pony,7.17,PLAYMOBIL,Nicht Zutreffend.,OneSize,70522,Polypropylen,Inhalt: 1 Stück


In [65]:
#check session data
Sessions_train.head()

,prev_items,next_item,locale
0,"[B09W9FND7K, B09JSPLN1M]",B09M7GY217,DE
1,"[B076THCGSG, B007MO8IME, B08MF65MLV, B001B4TKA0]",B001B4THSA,DE
2,"[B0B1LGXWDS, B00AZYORS2, B0B1LGXWDS, B00AZYORS...",B0767DTG2Q,DE
3,"[B09XMTWDVT, B0B4MZZ8MB, B0B7HZ2GWX, B09XMTWDV...",B0B4R9NN4B,DE
4,"[B09Y5CSL3T, B09Y5DPTXN, B09FKD61R8]",B0BGVBKWGZ,DE


we have the right formats

## The problem is in two formats
1. categorise low frequency products(maybe we can categorise products from mostly interracted with to less interracted with, then..)
2. using session data rank the most likely products to be interracted with

In [69]:
Sessions_train['prev_items'][0]


['B09W9FND7K', 'B09JSPLN1M']

In [ ]:
# feature engineering

### create and analyze product frequency

In [ ]:

class ProductDataset(Dataset):
    def __init__(self,
                 sessions_df: pd.DataFrame,
                 product_df: pd.DataFrame,
                 tokenizer: GPT2Tokenizer,
                 max_length: int = 512,
                 n_shots: int = 3):
        """
        Dataset for few-shot product categorization

        Args:
            sessions_df: DataFrame with prev_item (list) and next_item columns
            product_df: DataFrame with product attributes and categories
            tokenizer: GPT2 tokenizer
            max_length: Maximum sequence length
            n_shots: Number of examples to use in few-shot learning
        """
        self.sessions_df = sessions_df
        self.product_df = product_df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.n_shots = n_shots

        # Separate frequent and rare products
        self.product_counts = self._get_product_counts()
        self.frequent_products = self._get_frequent_products()
        self.rare_products = set(product_df.index) - self.frequent_products

        # Prepare few-shot examples
        self.shot_examples = self._prepare_shot_examples()

        # Prepare data for training
        self.data = self._prepare_data()

    def _get_product_counts(self) -> Dict[str, int]:
        """Count product occurrences in sessions"""
        counts = {}
        for _, row in self.sessions_df.iterrows():
            for item in row['prev_items']:
                counts[item] = counts.get(item, 0) + 1
            counts[row['next_item']] = counts.get(row['next_item'], 0) + 1
        return counts

    def _get_frequent_products(self, min_count: int = 5) -> set:
        """Get set of frequent products"""
        return {pid for pid, count in self.product_counts.items()
                if count >= min_count}

    def _prepare_shot_examples(self) -> Dict[str, List[str]]:
        """Prepare few-shot examples for each category"""
        examples = {}
        for category in self.product_df['category'].unique():
            # Get frequent products in this category
            category_products = self.product_df[
                (self.product_df['category'] == category) &
                (self.product_df.index.isin(self.frequent_products))
            ]

            # Select random examples
            selected = category_products.sample(
                n=min(self.n_shots, len(category_products))
            )

            examples[category] = [
                self._format_product_text(idx, row)
                for idx, row in selected.iterrows()
            ]
        return examples

    def _format_product_text(self, product_id: str, product_row: pd.Series) -> str:
        """Format product information as text"""
        return (f"Product: {product_row['title']}, "
                f"Price: {product_row['price']}, "
                f"Brand: {product_row['brand']}, "
                f"Color: {product_row['color']}, "
                f"Category: {product_row['category']}")

    def _prepare_data(self) -> List[Dict]:
        """Prepare data for training"""
        data = []
        for product_id in self.rare_products:
            product_row = self.product_df.loc[product_id]

            # Get relevant sessions
            relevant_sessions = self.sessions_df[
                (self.sessions_df['prev_items'].apply(lambda x: product_id in x)) |
                (self.sessions_df['next_item'] == product_id)
            ]

            # Prepare input text with few-shot examples and target product
            shot_texts = []
            for category in self.shot_examples:
                shot_texts.extend(self.shot_examples[category])

            target_text = self._format_product_text(product_id, product_row)

            # Combine into final text
            text = "\n".join(shot_texts + [target_text])

            data.append({
                'product_id': product_id,
                'text': text,
                'category': product_row['category']
            })
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        # Tokenize text
        encoding = self.tokenizer(
            item['text'],
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'product_id': item['product_id'],
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'category': item['category']
        }

class ProductCategorizer(nn.Module):
    def __init__(self,
                 num_categories: int,
                 pretrained_model: str = 'gpt2',
                 hidden_size: int = 768):
        """
        Few-shot product categorization model

        Args:
            num_categories: Number of product categories
            pretrained_model: Name of pretrained model to use
            hidden_size: Size of hidden layers
        """
        super().__init__()
        self.gpt = GPT2Model.from_pretrained(pretrained_model)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size, num_categories)
        )

    def forward(self, input_ids, attention_mask):
        # Get GPT2 embeddings
        outputs = self.gpt(input_ids, attention_mask=attention_mask)
        # Use last hidden state of [CLS] token for classification
        pooled_output = outputs.last_hidden_state[:, 0, :]
        # Classify
        logits = self.classifier(pooled_output)
        return logits

def train_categorizer(train_loader: DataLoader,
                     model: ProductCategorizer,
                     num_epochs: int = 5,
                     learning_rate: float = 2e-5,
                     device: str = 'cuda'):
    """Train the categorizer model"""
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['category'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f'Epoch {epoch+1}, Average Loss: {avg_loss:.4f}')

    return model

In [ ]:


class ProductDataset(Dataset):
    def __init__(self,
                 sessions_df: pd.DataFrame,
                 product_df: pd.DataFrame,
                 tokenizer: GPT2Tokenizer,
                 n_categories: int = 10,
                 max_length: int = 512,
                 n_shots: int = 3):
        """
        Dataset for few-shot product categorization

        Args:
            sessions_df: DataFrame with prev_item (list) and next_item columns
            product_df: DataFrame with product attributes (title, price, brand, color)
            tokenizer: GPT2 tokenizer
            n_categories: Number of categories to generate
            max_length: Maximum sequence length
            n_shots: Number of examples to use in few-shot learning
        """
        self.sessions_df = sessions_df
        self.product_df = product_df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.n_shots = n_shots
        self.n_categories = n_categories

        # Process data and generate categories
        self._process_data()

    def _process_data(self):
        """Process data and generate categories"""
        # Count product occurrences
        self.product_counts = self._get_product_counts()
        self.frequent_products = self._get_frequent_products()
        self.rare_products = set(self.product_df.index) - self.frequent_products

        # Generate embeddings and categories for frequent products
        self.product_embeddings = self._create_product_embeddings()
        self.product_categories = self._generate_categories()

        # Update product_df with categories
        self.product_df['category'] = pd.Series(self.product_categories)

        # Prepare few-shot examples
        self.shot_examples = self._prepare_shot_examples()

        # Prepare data for training
        self.data = self._prepare_data()

    def _get_product_counts(self) -> Dict[str, int]:
        """Count product occurrences in sessions"""
        counts = {}
        for _, row in self.sessions_df.iterrows():
            for item in row['prev_items']:
                counts[item] = counts.get(item, 0) + 1
            counts[row['next_item']] = counts.get(row['next_item'], 0) + 1
        return counts

    def _get_frequent_products(self, min_count: int = 5) -> set:
        """Get set of frequent products"""
        return {pid for pid, count in self.product_counts.items()
                if count >= min_count}

    def _create_product_embeddings(self) -> Dict[str, np.ndarray]:
        """Create embeddings from product attributes and session data"""
        embeddings = {}

        # Create attribute embeddings
        for idx, row in self.product_df.iterrows():
            # Normalize numerical features
            numerical = np.array([float(row['price'])])
            numerical_normalized = (numerical - np.mean(numerical)) / np.std(numerical)

            # One-hot encode categorical features
            brand_enc = pd.get_dummies([row['brand']])
            color_enc = pd.get_dummies([row['color']])

            # Combine features
            embedding = np.concatenate([
                numerical_normalized,
                brand_enc.values.flatten(),
                color_enc.values.flatten()
            ])

            embeddings[idx] = embedding

        # Add session information for frequent products
        for pid in self.frequent_products:
            if pid in embeddings:
                # Find co-occurring products
                co_occurrences = []
                for _, row in self.sessions_df.iterrows():
                    if pid in row['prev_items'] or pid == row['next_item']:
                        co_occurrences.extend(row['prev_items'])
                        co_occurrences.append(row['next_items'])

                if co_occurrences:
                    # Create co-occurrence vector
                    co_occ_counts = pd.Series(co_occurrences).value_counts()
                    co_occ_normalized = co_occ_counts / len(co_occurrences)

                    # Append to existing embedding
                    embeddings[pid] = np.concatenate([
                        embeddings[pid],
                        co_occ_normalized.values
                    ])

        return embeddings

    def _generate_categories(self) -> Dict[str, int]:
        """Generate categories for products using clustering"""
        # Prepare data for clustering
        frequent_embeddings = {pid: self.product_embeddings[pid]
                             for pid in self.frequent_products
                             if pid in self.product_embeddings}

        X = np.array(list(frequent_embeddings.values()))
        products = list(frequent_embeddings.keys())

        # Perform clustering
        kmeans = KMeans(n_clusters=self.n_categories, random_state=42)
        clusters = kmeans.fit_predict(X)

        # Create category mapping for frequent products
        categories = {pid: int(cluster) for pid, cluster in zip(products, clusters)}

        # Assign categories to rare products based on nearest neighbors
        for pid in self.rare_products:
            if pid in self.product_embeddings:
                # Find nearest cluster
                embedding = self.product_embeddings[pid]
                cluster = kmeans.predict([embedding])[0]
                categories[pid] = int(cluster)

        return categories

    def _prepare_shot_examples(self) -> Dict[int, List[str]]:
        """Prepare few-shot examples for each category"""
        examples = {}
        for category in range(self.n_categories):
            # Get frequent products in this category
            category_products = [pid for pid, cat in self.product_categories.items()
                               if cat == category and pid in self.frequent_products]

            # Select random examples
            if category_products:
                selected = np.random.choice(
                    category_products,
                    size=min(self.n_shots, len(category_products)),
                    replace=False
                )

                examples[category] = [
                    self._format_product_text(pid)
                    for pid in selected
                ]

        return examples

    def _format_product_text(self, product_id: str) -> str:
        """Format product information as text"""
        row = self.product_df.loc[product_id]
        return (f"Product: {row['title']}, "
                f"Price: {row['price']}, "
                f"Brand: {row['brand']}, "
                f"Color: {row['color']}, "
                f"Category: {self.product_categories[product_id]}")

    def _prepare_data(self) -> List[Dict]:
        """Prepare data for training"""
        data = []
        for product_id in self.rare_products:
            if product_id not in self.product_categories:
                continue

            # Get examples for the category
            category = self.product_categories[product_id]
            shot_texts = []
            for cat in self.shot_examples:
                shot_texts.extend(self.shot_examples[cat])

            # Add target product
            target_text = self._format_product_text(product_id)

            # Combine into final text
            text = "\n".join(shot_texts + [target_text])

            data.append({
                'product_id': product_id,
                'text': text,
                'category': category
            })

        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        encoding = self.tokenizer(
            item['text'],
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'product_id': item['product_id'],
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'category': torch.tensor(item['category'], dtype=torch.long)
        }

In [87]:
#reset index of product_train
Products_train.set_index('id',inplace=True)

# Initialize tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
# Create dataset
dataset = ProductDataset(Sessions_train, Products_train, tokenizer)

# Create dataloader
dataloader = DataLoader(
    dataset,
    batch_size=8,
    shuffle=True
)

In [ ]:
#Training
# Initialize model
model = ProductCategorizer(
    num_categories=len(Product_train['category'].unique())
)

# Train model
trained_model = train_categorizer(
    dataloader,
    model,
    num_epochs=5
)

In [ ]:
# Initialize model
model = ProductCategorizer(
    num_categories=len(Products_train['category'].unique())
)

# Train model
trained_model = train_categorizer(
    dataloader,
    model,
    num_epochs=5
)

## checkpoint the above code was stuck in a loop check suggestions.txt for next steps

In [ ]:
#Testing and Evaluation